In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jun-2021/train.csv", index_col ='id')
test = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv", index_col ='id')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")
submission.head()

In [ ]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

In [ ]:
train.target.value_counts()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:

train['target'] = train['target'].replace({'Class_1': 0, 'Class_2' : 1, 'Class_3' : 2, 'Class_4' : 3,
                                      'Class_5' : 4, 'Class_6' : 5, 'Class_7' : 6, 'Class_8' : 7,
                                      'Class_9' : 8})
train['target'] = train['target'].astype(int)

In [ ]:
train.target.value_counts()

In [ ]:
X = train.drop(['target'], axis = 1)
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks 
    
model = keras.Sequential([
        layers.BatchNormalization(input_shape = [75]),
        layers.Dense(units = 128, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(units = 64, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(units = 32, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),    
        layers.Dense(9, activation = 'softmax'),
    ])
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate=0.001), metrics='accuracy')

early_stopping = callbacks.EarlyStopping(
    patience=20,
    min_delta=0.0000001,
    restore_best_weights=True,
)

plateau = callbacks.ReduceLROnPlateau(
    factor = 0.5,                                     
    patience = 2,                                   
    min_delt = 0.000001,                                
    cooldown = 0,                               
    verbose = 1)

history = model.fit(X_train, y_train,
          batch_size = 128, epochs = 100,
          validation_data=(X_test, y_test),
          callbacks=[early_stopping, plateau]);


In [ ]:
score = model.evaluate(X_test, y_test)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}%'.format(score[1] * 100))

In [ ]:
pred = np.argmax(model.predict(X_test), axis=-1)
pred[:10]

In [ ]:
from sklearn.metrics import log_loss
pred_prob = model.predict_proba(X_test)
logloss = log_loss(y_test, pred_prob)
logloss

In [ ]:
test_pred= model.predict_proba(X_test)
test_pred

In [ ]:
test_pred=pd.DataFrame(test_pred)
test_pred

In [ ]:
test_pred.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']

In [ ]:
test_pred.head()

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
id_col= test_df['id']

In [ ]:
final_test= pd.concat([id_col,test_pred],
                      axis=1)
final_test.to_csv('submission.csv',index=False)

In [ ]:
submission.head()

In [ ]:
final_test.head()

# Please upvote if you like it, this gives us inspiration to work more.